In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

DATASET_PATH = "dataset"

image_size = 64
X = []
y = []

class_names = sorted([
    d for d in os.listdir(DATASET_PATH)
    if os.path.isdir(os.path.join(DATASET_PATH, d)) and not d.startswith(".")
])

for label_index, class_name in enumerate(class_names):
    class_folder = os.path.join(DATASET_PATH, class_name)
    if not os.path.isdir(class_folder):
        continue
    for filename in os.listdir(class_folder):
        filepath = os.path.join(class_folder, filename)
        img = cv2.imread(filepath, cv2.IMREAD_COLOR)
        if img is not None:
            img = cv2.resize(img, (image_size, image_size))
            X.append(img)
            y.append(label_index)

X = np.array(X)
y = np.array(y)

X_normalized = X / 255.0
n_samples = len(X_normalized)
X_flattened = X_normalized.reshape((n_samples, -1))

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X_flattened, y, test_size=0.2, stratify=y, random_state=42
)

# Train logistic regression
clf = LogisticRegression(max_iter=10000)
clf.fit(X_train, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test)


# Extension of the first part to process videos

This section extends the first part of the assessment by applying the trained logistic regression model to video frames. The goal is to process a video and detect whether a carrot appears in any frame.


This code processes a video file frame-by-frame using OpenCV, resizing each frame to the model’s expected input size, normalizing pixel values, and flattening the image data to feed into the pretrained logistic regression classifier. If a carrot is detected, the current timestamp is recorded and printed.

In [ ]:
import cv2
import numpy as np
from datetime import datetime

# Assuming clf and class_names are already loaded/trained

image_size = 64
target_class_name = "Carrot"
target_class_index = class_names.index(target_class_name)

video_path = "clip4.mov"
cap = cv2.VideoCapture(video_path)

detections = []

while True:
    ret, frame = cap.read()
    if not ret:
        break


    resized = cv2.resize(frame, (image_size, image_size))
    normalized = resized / 255.0
    flattened = normalized.reshape(1, -1)
    pred = clf.predict(flattened)[0]

    if pred == target_class_index:
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"Carrot detected at {timestamp}")
        detections.append(timestamp)
        # Optionally save frame
        cv2.imwrite(f"detected_carrot_{timestamp.replace(':','_')}.jpg", frame)

    # Display with label
    label = class_names[pred]
    color = (0, 255, 0) if pred == target_class_index else (0, 0, 255)
    cv2.putText(frame, f"Detected: {label}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    cv2.imshow("Video Carrot Detection", frame)

    # Press 'q' to quit early
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



For demo purposes, the code takes a prerecorded video. In order to use live video, change cv2.VideoCapture(video_path) to cv2.VideoCapture(0) to open default webcam.